# Narowcast Server service migration to Distribution Services

## 1. Getting data from NC

### 1.1 List of NC Services

In [ ]:
# Run this SQL code against Narrowcast Server database
"""
select 
names1.MR_OBJECT_ID AS serviceID, 
names1.MR_OBJECT_NAME AS service_name, 
parent1.MR_OBJECT_NAME AS foldername, 
names2.MR_OBJECT_NAME AS publication_name, 
names3.MR_OBJECT_NAME AS document_name, 
info3.MR_OBJECT_SUBTYPE AS doc_type, 
names4.MR_OBJECT_ID AS info_obj_id,
names4.MR_OBJECT_NAME AS info_obj_name, 
info4.MR_OBJECT_SUBTYPE AS info_obj_subtype   
from 
MSTROBJNAMES names1, 
MSTROBJINFO info1, 
MSTROBJNAMES parent1, 
MSTROBJDEPN dpns,
MSTROBJNames names2, 
MSTROBJDEPN dpns2,
MSTROBJNames names3,
MSTROBJINFO info3, 
MSTROBJDEPN dpns3,
MSTROBJNames names4,
MSTROBJInfo info4  
where names1.MR_OBJECT_ID = dpns.MR_INDEP_OBJID
and names1.MR_OBJECT_ID = info1.MR_OBJECT_ID
and info1.MR_PARENT_ID = parent1.MR_OBJECT_ID
and dpns.MR_DEPN_OBJID = names2.MR_OBJECT_ID
and names2.MR_OBJECT_ID = dpns2.MR_INDEP_OBJID
and dpns2.MR_DEPN_OBJID = names3.MR_OBJECT_ID
and names3.MR_OBJECT_ID = dpns3.MR_INDEP_OBJID
and names3.MR_OBJECT_ID = info3.MR_OBJECT_ID
and dpns3.MR_DEPN_OBJID = names4.MR_OBJECT_ID
and dpns3.MR_DEPN_OBJID = info4.MR_OBJECT_ID
and names1.MR_Object_Type = 19
and names2.MR_Object_Type = 16
and names3.MR_Object_Type = 14
and names4.MR_Object_Type = 4
and info4.MR_OBJECT_SubType <> 1
"""

<img src="Images/NC_services.png">

### 1.2 NC Service details

In [ ]:
"""
select  
names1.MR_OBJECT_ID AS serviceID, --This is Service ID
names1.MR_OBJECT_NAME AS service_name, 
names2.MR_OBJECT_NAME AS subset_name, 
a11.MR_ADD_DISPLAY AS dispname, 
a11.MR_PHYSICAL_ADD AS email, 
a13.MR_USER_NAME,
sp.MR_INFOSOURCE_ID, 
sp.MR_QUES_OBJ_ID, 
po.mr_seq, 
sp.MR_USER_PREF,
po.MR_PREF_OBJ
from 
MSTROBJNames names1,
MSTROBJINFO info1, 
MSTROBJDEPN dpns,
MSTROBJNames names2,

MSTRSUBSCRIPTIONS a12,
MSTRADDRESSES a11,
MSTRUSERS a13,
MSTRSUBPREF sp,
MSTRPREFOBJS po

where names1.MR_Object_Type = 19
and names2.MR_Object_Type = 17
and info1.MR_STATUS =1

and names1.MR_OBJECT_ID = info1.MR_OBJECT_ID
and names1.MR_OBJECT_ID = dpns.MR_INDEP_OBJID
and dpns.MR_DEPN_OBJID = names2.MR_OBJECT_ID
and names2.MR_OBJECT_ID = a12.MR_SUB_SET_ID
and a11.MR_ADDRESS_ID = a12.MR_ADDRESS_ID
and a12.MR_SUB_GUID = sp.MR_SUB_GUID
and sp.MR_PREF_OBJ_ID = po.MR_PREF_OBJ_ID
and a12.MR_USER_ID = a13.MR_USER_ID
and names1.MR_OBJECT_ID = '047886F8A7474F4A929EC6DD135F0A98' --Filter for Service ID
"""

<img src="Images/service_details.png">

In [ ]:
with open('narrowcast_emails.csv', encoding="utf8", newline='') as f:
    email_list = [x.strip() for x in f]

## Automate tasks in MicroStrategy

In [ ]:
from datetime import datetime

from mstrio.connection import Connection
from mstrio.distribution_services import Content, EmailSubscription
from mstrio.users_and_groups import list_users


#### Parameters ####
api_login, api_password = 'administrator', ''
base_url = 'Insert Env URL'
project_id = 'Insert Project ID'

conn = Connection(base_url,api_login,api_password)

### Get users' default addresses

In [ ]:
users = list_users(connection=conn)
default_addresses=[]

for u in users:
    if u.addresses:
        user_addresses = [[u.name, u.id, uad['value']] for uad in u.addresses if uad['isDefault']==True]
        default_addresses.extend(user_addresses)

### Create a list of recipients

In [ ]:
# From MSTR Metadata
for d in default_addresses:
    print(d)

In [ ]:
# From Narrowcast
for e in email_list:
    print(e)

In [ ]:
# Match Metadata with Narrowcast
matched_emails = [d[1] for d in default_addresses if d[2] in email_list]
for m in matched_emails:
    print(m)

### Create a subscription

In [ ]:
# create an email subscription
recipient_ids = matched_emails[:]
content_id = 'Insert Content ID'
schedule_id = 'Insert Schedule ID'
subscription_name = 'REST_API_'+datetime.now().strftime("%Y-%m-%d__%H-%M")
subject_txt='Email Subject'
message_txt="Message Text"
EmailSubscription.create(connection=conn,
                         name=subscription_name, 
                         project_id=project_id,
                         send_now = True,
                         contents=[Content(id=content_id, type='report', name='Report 1',
                                           personalization=Content.Properties(format_type='EXCEL'))],
                         schedules=[schedule_id],
                         recipients=recipient_ids,
                         email_subject=subject_txt,
                         email_message=message_txt,
                         email_send_content_as="data")
